# Vehicle DB Exploration

[Link documentation here](https://vehicledatabases.com/)

In [ ]:
import requests 

api_key="[INSERT YOUR API KEY]"

In [ ]:
import pandas as pd
import json
import sys

In [ ]:
start_year = 2019
end_year = 2025
vehicle_year_make_model = []
counter = 0

def get_vehicle_make_year(year):
    url = "https://api.vehicledatabases.com/market-value/options/v3/make/"+ str(year)
    payload={}
    headers = {
        'x-AuthKey': api_key
    }
    try:
        result = json.loads((requests.request("GET", url, headers=headers, data=payload)).text)
        if "status" in result and result["status"] == "success":
            return result
        else:
            print(result)
    except ValueError:
        print(ValueError)

def get_vehicle_model_make_year(year, make):
    url = "https://api.vehicledatabases.com/market-value/options/v3/model/" + str(year) + "/" + make
    payload={}
    headers = {
        'x-AuthKey': api_key
    }
    try:
        result = json.loads((requests.request("GET", url, headers=headers, data=payload)).text)
        if "status" in result and result["status"] == "success":
            return get_vehicle_model_make_year_electric(result, year, make)
        else:
            print(result)
    except ValueError:
        print(ValueError)

def get_vehicle_model_make_year_electric(result_vehicle_model_make_year, year, make):
    url = "https://api.vehicledatabases.com/electric-vehicle/options/v2/model/" + str(year) + "/" + make
    payload={}
    headers = {
        'x-AuthKey': api_key
    }
    try:
        result_vehicle_model_make_year_electric = json.loads((requests.request("GET", url, headers=headers, data=payload)).text)
        if "status" in result_vehicle_model_make_year_electric and result_vehicle_model_make_year_electric["status"] == "success":
            if "data" in result_vehicle_model_make_year_electric and len(result_vehicle_model_make_year_electric["data"]) > 0:
                result_vehicle_model_make_year["data"]["electric_models"] = result_vehicle_model_make_year_electric["data"]
            else:
                result_vehicle_model_make_year["data"]["electric_models"] = []
            return result_vehicle_model_make_year
        else:
            print(result_vehicle_model_make_year_electric)
    except ValueError:
        print(ValueError)

for year in range(start_year, end_year):
    counter = 0
    result_year_make = get_vehicle_make_year(year)
    if "data" in result_year_make:
        year = str(result_year_make["data"]["year"])
        makes = result_year_make["data"]["makes"]
        for make in makes:
            counter = counter + 1
            result_year_make_model = get_vehicle_model_make_year(year, make)
            models = result_year_make_model["data"]["models"]
            electric_models = result_year_make_model["data"]["electric_models"]
            for model in models:
                sys.stdout.write(f"\rRunning... now for brand number ({counter}/{len(makes)}) position at year {year}")
                sys.stdout.flush() 
                if model in electric_models:
                    is_model_electric = True
                else:
                    is_model_electric = False
                vehicle_year_make_model.append({
                    "year": year
                    , "make": make
                    , "model": model
                    , "is_model_electric": is_model_electric
                })

vehicle_year_make_model_df = pd.DataFrame(vehicle_year_make_model)
print("DONE!")

In [ ]:
vehicle_year_make_model_df

In [ ]:
import duckdb

with duckdb.connect("vehicle_database.db") as con:
    try:
        con.sql("CREATE TABLE vehicle_year_make_model AS SELECT * FROM vehicle_year_make_model_df")
    except ValueError:
        print(ValueError)

In [ ]:
import pandas as pd
import json
import sys

vehicle_year_make_model_market_value = []
counter = 0

con = duckdb.connect("vehicle_database.db")
results = con.sql("select distinct year, make, model from vehicle_year_make_model").df()

def get_vehicle_year_make_model_market_value(year, make, model):
    url = "https://api.vehicledatabases.com/market-value/v2/ymm/" + str(year) + "/" + make + "/" + model
    payload={}
    headers = {
        'x-AuthKey': api_key
    }
    try:
        result = requests.request("GET", url, headers=headers, data=payload)
        if result["status"] == "success":
            return result.text
        else:
            print(result)
    except ValueError:
        print(ValueError)

for index, row in results.iterrows():
    counter = 0
    year = row["year"]
    make = row["make"]
    model = row["model"]
    
    result_text = get_vehicle_year_make_model_market_value(year, make, model)
    result = json.loads(result_text)

    if "data" in result:
        counter = counter + 1
        sys.stdout.write(f"\rRunning... now for brand number ({counter}/{len(makes)}) position at year {year}")
        sys.stdout.flush() 

        market_value_datas = result["data"]["market_value"]["market_value_data"]

        for market_value_data in market_value_datas:
            trim = market_value_data["trim"]
            market_value_details = market_value_data["market value"]

            for market_value_detail in market_value_details:
                condition = market_value_detail["Condition"]
                trade_in = market_value_detail["Trade-In"]
                private_party = market_value_detail["Private Party"]
                dealer_detail = market_value_detail["Dealer Retail"]

                vehicle_year_make_model_market_value.append({
                    "year": year
                    , "make": make
                    , "model": model
                    , "trim": trim
                    , "condition": condition
                    , "trade_in": trade_in
                    , "private_party": private_party
                    , "dealer_detail": dealer_detail
                })
    else:
        continue

vehicle_year_make_model_value_df = pd.DataFrame(vehicle_year_make_model_market_value)
print("DONE")
con.close()

In [ ]:
with duckdb.connect("vehicle_database.db") as con:
    results = con.sql("select * from vehicle_year_make_model_value").df()

results

In [ ]:
import duckdb

sql_script = '''
    show tables
'''

with duckdb.connect("dbt_workspace/vehicle_database/data/vehicle_database.db") as con:
    temp_df = con.sql(sql_script)#.df()
    display(temp_df)